# Data Cleaning and Preparation
## 1、处理缺失值
### - 去除与填充
## 2、数据转换
### - 去重、映射函数（函数，序列值）、数据替换、索引和列名称的重命名、离散化和分区间binning、检查异常值、排列与随机抽样、计算指标与虚拟变量变换
## 3、字符串操作
### - 字符串对象方法、正则表达式

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

In [5]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [6]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [26]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data
cleaned

,0,1,2
0,1.0,6.5,3.0


### how属性强调是否一行都为空才删除空值

In [9]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


### 当axis=1时，如果是求平均，那么是从左到右横向求平均；如果是拼接，那么也是左右横向拼接；如果是drop，那么也是横向发生变化，体现为列的减少
### axis=1代表横向，从左向右，axis=0代表纵向，从上到下,默认为0

In [13]:
data[4] = NA
data
data.dropna(axis=0, how='all')

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


 ### 简单的理解：这一行除去NA值，剩余数值的数量大于等于n，便显示这一行

In [27]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA

df.dropna()
df
df.dropna(thresh=2)

,0,1,2
2,-1.677790,NaN,-1.563740
3,-0.367488,NaN,1.219954
4,-0.247699,-0.416232,-0.116747
5,-1.844788,2.068708,-0.776967
6,1.440167,-0.110557,1.227387


### Filling In Missing Data

In [28]:
df.fillna(0)

,0,1,2
0,-0.017141,0.000000,0.000000
1,-0.757177,0.000000,0.000000
2,-1.677790,0.000000,-1.563740
3,-0.367488,0.000000,1.219954
4,-0.247699,-0.416232,-0.116747
5,-1.844788,2.068708,-0.776967
6,1.440167,-0.110557,1.227387


In [29]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.017141,0.500000,0.000000
1,-0.757177,0.500000,0.000000
2,-1.677790,0.500000,-1.563740
3,-0.367488,0.500000,1.219954
4,-0.247699,-0.416232,-0.116747
5,-1.844788,2.068708,-0.776967
6,1.440167,-0.110557,1.227387


In [30]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.017141,0.000000,0.000000
1,-0.757177,0.000000,0.000000
2,-1.677790,0.000000,-1.563740
3,-0.367488,0.000000,1.219954
4,-0.247699,-0.416232,-0.116747
5,-1.844788,2.068708,-0.776967
6,1.440167,-0.110557,1.227387


In [33]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df
df.fillna(method='ffill')
df.fillna(method='ffill', limit=2)

,0,1,2
0,-1.168634,-0.825020,-2.644409
1,-0.152986,-0.751921,-0.132609
2,1.457300,-0.751921,-0.493779
3,1.239980,-0.751921,1.430042
4,-0.846852,NaN,1.430042
5,-0.255491,NaN,1.430042


In [34]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation

### Removing Duplicates

In [35]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [36]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [37]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [42]:
data['v1'] = range(7)
data
#data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [43]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [44]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [45]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [47]:
lowercased = data['food'].str.lower()
lowercased
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [23]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values

In [48]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [49]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [50]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [51]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [52]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [54]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [56]:
transform = lambda x: x[:4].upper()
transform
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [57]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [58]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [59]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [60]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

In [61]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [62]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [65]:
cats.codes
cats.categories
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [66]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [67]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [40]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.34, 0.55], (0.34, 0.55], (0.76, 0.97], (0.76, 0.97], (0.34, 0.55], ..., (0.34, 0.55], (0.34, 0.55], (0.55, 0.76], (0.34, 0.55], (0.12, 0.34]]
Length: 20
Categories (4, interval[float64]): [(0.12, 0.34] < (0.34, 0.55] < (0.55, 0.76] < (0.76, 0.97]]

### qcut使得每个箱子里的个数相同

In [68]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats
pd.value_counts(cats)

(0.57, 3.928]                    250
(-0.0453, 0.57]                  250
(-0.675, -0.0453]                250
(-2.9499999999999997, -0.675]    250
dtype: int64

In [69]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.229, -0.0453], (-2.9499999999999997, -1.229], (-0.0453, 1.254], (-0.0453, 1.254], (-0.0453, 1.254], ..., (-0.0453, 1.254], (-0.0453, 1.254], (-0.0453, 1.254], (-2.9499999999999997, -1.229], (-0.0453, 1.254]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -1.229] < (-1.229, -0.0453] < (-0.0453, 1.254] < (1.254, 3.928]]

### Detecting and Filtering Outliers

### np.arange()函数返回一个有终点和起点的固定步长的排列,一个参数从0开始步长为1，两个参数起点和终点，三个参数，起点终点和步长

### rand函数根据给定维度生成[0,1)之间的数据，randn生成给定维度符合标准正态分布的数据

In [70]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.024645,-0.007512,-0.040121,0.056530
std,1.010489,1.005025,0.999603,1.007002
min,-3.333767,-3.745356,-3.428254,-3.645860
25%,-0.612162,-0.701202,-0.740152,-0.598600
50%,-0.013609,-0.014664,-0.082368,0.059991
75%,0.688640,0.700701,0.632001,0.772359
max,3.525865,2.735527,3.366626,2.763474


In [71]:
col = data[2]
col[np.abs(col) > 3]

217   -3.428254
594    3.366626
Name: 2, dtype: float64

In [72]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
0,-0.025907,-3.399312,-0.974657,-0.685312
19,3.260383,0.963301,1.201206,-1.852001
95,-0.196713,-3.745356,-1.520113,-0.346839
194,-3.056990,1.918403,-0.578828,1.847446
217,0.326045,0.425384,-3.428254,-0.296336
281,-3.184377,1.369891,-1.074833,-0.089937
502,0.208011,-0.150923,-0.362528,-3.548824
594,0.193299,1.397822,3.366626,-2.372214
741,3.525865,0.283070,0.544635,0.462204
761,-0.450721,-0.080332,0.599947,-3.645860


### np.sign将大于0的返回1，乘以3也就是大于3的数值显示为3

In [48]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.050286,0.025567,-0.001399,-0.051765
std,0.992920,1.004214,0.991414,0.995761
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.599807,-0.612162,-0.687373,-0.747478
50%,0.047101,-0.013609,-0.022158,-0.088274
75%,0.756646,0.695298,0.699046,0.623331
max,2.653656,3.000000,2.735527,3.000000


In [73]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0
2,-1.0,1.0,-1.0,-1.0
3,-1.0,-1.0,1.0,-1.0
4,-1.0,1.0,-1.0,-1.0


### Permutation and Random Sampling
### 排列与随机抽样

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df
sampler = np.random.permutation(5)
sampler

array([4, 1, 3, 2, 0])

In [3]:
df
df.take(sampler)

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
3,12,13,14,15
2,8,9,10,11
0,0,1,2,3


In [76]:
a=np.random.permutation(df)
a

array([[ 4,  5,  6,  7],
       [ 0,  1,  2,  3],
       [12, 13, 14, 15],
       [ 8,  9, 10, 11],
       [16, 17, 18, 19]])

In [4]:
df.sample(n=3)

,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7


In [6]:
choices = pd.Series([5, 7, -1, 6, 4])
choices
draws = choices.sample(n=10, replace=True)
draws

4    4
4    4
2   -1
4    4
1    7
3    6
3    6
4    4
0    5
1    7
dtype: int64

### Computing Indicator/Dummy Variables
### 计算指标/虚拟变量

In [3]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [6]:
import pandas as pd
dummies = pd.get_dummies(df['key'], prefix='key')
dummies
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [7]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [8]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [9]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [12]:
import numpy as np
zero_matrix = np.zeros((len(movies), len(genres)))
zero_matrix
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
gen = movies.genres[0]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [64]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [65]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
                                ...             
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [16]:
np.random.seed(12345)
values = np.random.rand(10)
values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation
### 字符串操作

### String Object Methods

In [17]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [18]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [19]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [20]:
'::'.join(pieces)

'a::b::guido'

In [28]:
'guido' in pieces
val
val.index('b')
val.find(':')

-1

In [29]:
val.index(':')

ValueError: substring not found

In [30]:
val.count(',')

2

In [32]:
val.replace(',', '::')
val.replace(',', '')

'ab  guido'

### Regular Expressions

### \s表示匹配任何空白字符，包括空格、制表符、换页符等等, 等价于[ \f\n\r\t\v]而"\s+"则表示匹配任意多个上面的字符

In [36]:
import re
text = "foo    bar\t   baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [35]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [37]:
regex.findall(text)

['    ', '\t   ', '  \t']

In [41]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)
regex.split(text)

['Dave ', '\nSteve ', '\nRob ', '\nRyan ', '\n']

In [44]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [43]:
m = regex.search(text)
m
text[m.start():m.end()]

'dave@google.com'

In [45]:
print(regex.match(text))

None


In [85]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [86]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [87]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [88]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [89]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### Vectorized String Functions in pandas

In [47]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [48]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [49]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [dave@google.com]
Steve    [steve@gmail.com]
Rob        [rob@gmail.com]
Wes                    NaN
dtype: object

In [53]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
type(matches)

pandas.core.series.Series

In [52]:
matches.str.get(1)
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [54]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [55]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

NameError: name 'PREVIOUS_MAX_ROWS' is not defined

## Conclusion